In [1]:
import pandas as pd 
import numpy as  np 

data = {'sender'          :[5,1,2,2,3,3,1],
        'receiver'        :[2,3,1,3,1,2,4],
        'amount'          :[10,15,20,25,20,15,5],
        'transaction_date':['12-feb-20',
                            '13-feb-20',
                            '13-feb-20',
                            '14-feb-20',
                            '15-feb-20',
                            '15-feb-20',
                            '16-feb-20']
        }

transactions = pd.DataFrame(data)
transactions

,sender,receiver,amount,transaction_date
0,5,2,10,12-feb-20
1,1,3,15,13-feb-20
2,2,1,20,13-feb-20
3,2,3,25,14-feb-20
4,3,1,20,15-feb-20
5,3,2,15,15-feb-20
6,1,4,5,16-feb-20


In [11]:
debited  = transactions[['sender','amount']].groupby('sender').sum()
credited = transactions[['receiver','amount']].groupby('receiver').sum()

debited.reset_index(inplace  = True)
credited.reset_index(inplace = True)

debited.rename(columns  = {'sender'  :'user_id','amount':'debited'},  inplace = True)
credited.rename(columns = {'receiver':'user_id','amount':'credited'}, inplace = True)

In [12]:
debited

,user_id,debited
0,1,20
1,2,45
2,3,35
3,5,10


In [13]:
credited

,user_id,credited
0,1,40
1,2,25
2,3,40
3,4,5


In [14]:
changes = pd.merge(debited, credited, on = 'user_id',how = 'outer')
changes

,user_id,debited,credited
0,1,20.0,40.0
1,2,45.0,25.0
2,3,35.0,40.0
3,5,10.0,NaN
4,4,NaN,5.0


In [16]:
changes['debited']  = changes['debited'].replace(np.nan, 0)
changes['credited'] = changes['credited'].replace(np.nan,0)
changes

,user_id,debited,credited
0,1,20.0,40.0
1,2,45.0,25.0
2,3,35.0,40.0
3,5,10.0,0.0
4,4,0.0,5.0


In [19]:
changes['net_change'] = changes['credited'] - changes['debited']
changes[['user_id','net_change']].sort_values(by = 'net_change',ascending = False)

,user_id,net_change
0,1,20.0
2,3,5.0
4,4,5.0
3,5,-10.0
1,2,-20.0
